Import necessary packages

In [1]:
# import necessary packages

import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tvt

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from functools import reduce
from functools import partial

os.chdir("..")

from timeit import default_timer
import scipy.io
import sys
from itertools import chain
import skfmm
import time

from models.TrainPlanningOperator4D import PlanningOperator4D, smooth_chi
from models.utilities import *
from scipy.io import loadmat
from planner import getPNOValueFunction, perform_gradient_descent, generaterandompos, getFMMValueFunction


Loading the Trained Model and Maps

In [2]:
maps = loadmat('dataset/occupancyGridsSphere_matlab.mat') 
occupancygrids = 1 - maps['occupancyGrids']
obstaclepositions = maps['blockPositions']
testmaps = occupancygrids#Unseen Maps
testobstaclepositions = obstaclepositions
goalpos = generaterandompos(testmaps)
startpos = generaterandompos(testmaps)

modes = 3
width = 7
nlayers = 1

model = PlanningOperator4D(modes, modes, modes, modes, width, nlayers)
model.load_state_dict(torch.load("dataset/manipulator/planningoperator_manipulator17_m3_w8_l1_b10_lr3e-3_10g_20nov/n400_lr1.000000e-02_gamma6.000000e-01_wd3.000000e-06_seed5/model4d.ckpt"))
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

PlanningOperator4D(
  (fc0): Linear(in_features=4, out_features=7, bias=True)
  (conv0): SpectralConv4d()
  (w0): Conv4d(
    (conv3d_layers): ModuleList(
      (0): Conv3d(7, 7, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
    )
  )
  (fc1): DeepNormMetric(
    (Us): ModuleList(
      (0-1): 2 x Linear(in_features=7, out_features=128, bias=False)
    )
    (Ws): ModuleList(
      (0): ConstrainedLinear(in_features=128, out_features=128, bias=False)
    )
    (activation): MaxReLUPairwiseActivation(
      (avg_pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    )
    (output_activation): ConcaveActivation()
    (reduce_metric): ReduceMetric()
  )
)

In [3]:
def generate_cfg_trajectory(path, num_divisions=17):
    """Convert a path array to a cfg_trajectory dictionary."""
    
    # Generate joint range (-π to π) divided into num_divisions
    joint_range = np.linspace(-np.pi, np.pi, num_divisions)

    # Initialize trajectory dictionary for the 4 DOF joints
    cfg_trajectory = {
        'joint1': [],
        'joint2': [],
        'joint3': [],
        'joint4': []
    }

    # Map indices to joint angles and populate the cfg_trajectory dictionary
    for state in path:
        cfg_trajectory['joint1'].append(joint_range[int(state[0])])
        cfg_trajectory['joint2'].append(joint_range[int(state[1])])
        cfg_trajectory['joint3'].append(joint_range[int(state[2])])
        cfg_trajectory['joint4'].append(joint_range[int(state[3])])

    return cfg_trajectory

Compute Value Function and Generate Trajectories

In [71]:
test_idx = 14
test_idx = test_idx - 1

teststart = np.array([0, 3, 5, 8])
testgoal = np.array([8, 3, 5, 8])

testmap = testmaps[test_idx,:,:,:,:].squeeze()
testobstaclepositions = obstaclepositions[test_idx,:,:]
valuefunction =  getPNOValueFunction(testmap, testgoal, model)
print(testobstaclepositions)
success, path_length, path = perform_gradient_descent(valuefunction, teststart, testgoal)
print("Success:",success, "Path Length:", path_length)


[[ 0.12705108  0.11793257  0.05772725]
 [-0.04855625  0.12463218  0.01313024]]
Success: True Path Length: 12.292528739883945


In [72]:

path_urdfpy = np.array(path)
from urdfpy import URDF, Link, Joint, Visual, Collision, Geometry, Sphere, Material
from urdfpy import xyz_rpy_to_matrix
import matplotlib.animation as animation


# Load the original robot URDF
robot = URDF.load('robot/open_manipulator/open_manipulator_x_description/urdf/open_manipulator_x_robot.urdf')

# Scaling factor used in MATLAB
urdfpy_scaling = 2.0

# Define obstacle positions and radii
obstacle_positions = [
    [testobstaclepositions[0,0]/urdfpy_scaling, testobstaclepositions[0,1]/urdfpy_scaling, testobstaclepositions[0,2]/urdfpy_scaling],   # (x, y, z)
    [testobstaclepositions[1,0]/urdfpy_scaling, testobstaclepositions[1,1]/urdfpy_scaling, testobstaclepositions[1,2]/urdfpy_scaling]
]
obstacle_radii = [0.08, 0.08]  # Radii of spheres

# Define obstacle colors (RGBA)
obstacle_colors = [
    [1.0, 0.0, 0.0, 1.0],  # Red
    [0.0, 0.0, 1.0, 1.0]   # Blue
]

obstacle_links = []
obstacle_joints = []

base_link = "world"

# Create obstacle links and joints
for i, (pos, radius) in enumerate(zip(obstacle_positions, obstacle_radii)):
    sphere_geom = Geometry(sphere=Sphere(radius=radius))
    origin = pos + [0, 0, 0]  # No rotation

    material = Material(name=f"obstacle_color_{i}", color=obstacle_colors[i])
    visual = Visual(name=f"obstacle_{i}_visual", geometry=sphere_geom, origin=origin, material=material)
    collision = Collision(name=f"obstacle_{i}_collision", geometry=sphere_geom, origin=origin)

    obstacle_link = Link(name=f"obstacle_{i}", inertial=None, visuals=[visual], collisions=[collision])
    obstacle_joint = Joint(
        name=f"obstacle_{i}_joint",
        joint_type="fixed",
        parent=base_link,
        child=obstacle_link.name,
        origin=origin
    )

    obstacle_links.append(obstacle_link)
    obstacle_joints.append(obstacle_joint)

# Define manipulator material color (green)
manipulator_material = Material(name="manipulator_color", color=[0.2, 0.8, 0.2, 1.0])

# Rebuild each link with new visual materials
colored_links = []
for link in robot.links:
    new_visuals = []
    for visual in link.visuals:
        new_visual = Visual(
            geometry=visual.geometry,
            origin=visual.origin,
            material=manipulator_material
        )
        new_visuals.append(new_visual)

    # Reconstruct the link with updated visuals
    colored_link = Link(
        name=link.name,
        inertial=link.inertial,
        visuals=new_visuals,
        collisions=link.collisions
    )
    colored_links.append(colored_link)

# Construct the new URDF
new_urdf = URDF(
    name="robot_with_colored_obstacles",
    links=colored_links + obstacle_links,
    joints=robot.joints + obstacle_joints
)

# Generate your trajectory here
# Make sure the `generate_cfg_trajectory(path)` function is defined elsewhere
cfg_trajectory = generate_cfg_trajectory(path_urdfpy)

# Animate the URDF
new_urdf.animate(cfg_trajectory=cfg_trajectory)



Old stuff